In [20]:
def load_my_modules(prj_name=None, lst=None):

    import sys, os

    if prj_name==None:  prj_name = 'project_KnowledgeBase'       
    if lst==None:       lst = [
            '\\my_methods',
            '\\personal_expenses\\modules'
            # '\\qqq,',        
    ]

    # Получить текущую папку
    cwd = os.getcwd()
    prj_len = len(prj_name)
    i1 = cwd.find(prj_name)
    prj_dir = cwd[0:i1+prj_len]
    # d:\IDE_prj\project_KnowledgeBase

    # '\' -> '\\'
    if not prj_dir.find('\\\\'):
        prj_dir = prj_dir.replace('\\', '\\\\')
    
    # Проверить наличие модулей в sys.path_hook
    # Если отсутствют - добавить
    for module_dir in lst:
        module_dir = prj_dir + module_dir
        if not module_dir in sys.path:
            sys.path.insert(0, module_dir)
    return sys.path

load_my_modules()

['d:\\IDE_prj\\project_KnowledgeBase\\personal_expenses\\modules',
 'd:\\IDE_prj\\project_KnowledgeBase\\my_methods',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']

In [21]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import my_methods as mm

file_name = 'data/test_rnkb.xml'
xlm = ET.parse(file_name)
tag_root = xlm.getroot()
for tag in tag_root:
    print(tag)

level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'


<Element '{urn:schemas-microsoft-com:office:office}DocumentProperties' at 0x000001DE1F04A040>
<Element '{urn:schemas-microsoft-com:office:excel}ExcelWorkbook' at 0x000001DE1F04A180>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Styles' at 0x000001DE1F04A130>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet' at 0x000001DE1F04AD60>


In [5]:
# Print rows
for i_row, tag_row in enumerate(tag_root.findall(level)):
    print('row ', i_row, '-----------------', tag_row)
    for text in tag_row.itertext():
        print(text)

row  0 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F0520E0>
Сформировано 26.04.2022 08:56 GMT +03:00                                                                                                            Страница 1 из 3
row  1 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F052360>
Александр Квашенинников Геннадьевич
row  2 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F052540>
Московская обл, Балашиха г, Салтыковка мкр., 1-й
row  3 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F052680>
Тенистый проезд, д. 2, кв. 6
row  4 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F0527C0>
Выписка по карте Карта Дебетовая MASS N 2200 02** **** 2043
row  5 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001DE1F052950>
включая 

In [26]:
text = 'Расходы за период всего:                                               461 698.81 RUR'
n=len(text)
mm.text_to_float(text[24:n-4])

461698.81

In [ ]:
import datetime

def load_payments_from_XML_RNKB(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)
    # res['success'] = True - флаг успешного чтения формата файла

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [None, None,0,0]  # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]


    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    rows = []#{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

    flag_mark = False  # Найти текст 'Карта'
    flag_card = False  # Найти метку карты
    for i_row, tag_row in enumerate(tag_root.findall(level)):
        
        # flag1_mark  Найти метку в строке с заданным номером
        # Пример строки:
        #   Выписка по карте Карта Дебетовая MASS N 2200 02** **** 2043
        mark_text = 'Выписка по карте'
        mark_len = len(mark_text)
        if i_row == 4:


            for text in tag_row.itertext():
                if len(text) > mark_len:
                    if text[0:mark_len] != mark_text:
                        flag_mark = True

                        # Найти метку карты (в той же строке)
                        # res['card_mark']
                        res['card_mark'] = text[mark_len+1: len(text)] # 'Карта Дебетовая MASS N 2200 02** **** 2043'
                        flag_card = True        # допуск на дальнейший парсинг

            if not flag_mark:
                res.error_text = 'При попытке чтения XML файла в строке 4 не найдена метка "Карта" (код 001)'
                res['success'] = False
                return res

        # res['period']
        elif i_row == 6:
            # д.б. единственная строка в цикле
            for text in tag_row.itertext():
                res['period'] = text    # за период с 01.04.2022 по 26.04.2022
                if len(text) == 36: # len(text)=36
                    t1 = text[12:22]
                    t2 = text[26:36]
                    res['d1'] = datetime.datetime.strptime(t1, '%d.%m.%Y')
                    res['d2'] = datetime.datetime.strptime(t2, '%d.%m.%Y') + datetime.timedelta(days=1)
                else:
                    res.error_text = 'При попытке чтения XML файла в строке 4 не найдена метка "Карта" (код 002)'
                    res['success'] = False
                    return res
        
        # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]
        # 19 416.61 RUR    // "Поступления"
        elif i_row == 8: 
            val = None
            for i, text in enumerate(tag_row.itertext()):
                n = len(text)
                if n > 5:
                    val = mm.text_to_float(text[0:n-4])
            if val == None:
                res.error_text = 'При попытке чтения XML файла в строке 8 ошибка формата чтени поля "Поступления" (код 003)'
                res['success'] = False
                return res
            res['prop'][3] = val

        # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]
        # Расходы за период всего:                                               461 698.81 RUR
        elif i_row == 9: 
            val = None
            for i, text in enumerate(tag_row.itertext()):
                n = len(text)
                if n > 5:
                    val = mm.text_to_float(text[24:n-4])
            if val == None:
                res.error_text = 'При попытке чтения XML файла в строке 9 ошибка формата чтени поля "Расходы" (код 004)'
                res['success'] = False
                return res
            res['prop'][2] = val

        elif i_row >= 19:        
            # print('row ', i_row, '-----------------', tag_row)

            row = []
            for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
                if i_cel > 7: break
                text = ''
                for t in tag.itertext():
                    text = t
                row.append(text)
            # row.insert(0, len(row))
            if len(row) == 8:
                rows.append(row)


    # Обработать rows в массив записей rows_ar
    len1 = len(rows)
    len2 = len1//2
    if len1%2 != 0:
        res.error_text = f'При попытке чтения XML файла считано не четное количество строк в таблице: {len1}'
        return res
    rows2 = []

    #   Date	    Time  ...   Name	...	         Amount	....	Balance	Amount_num	Balance_num
    # 0	2022-03-31	15:44		Перевод с карты		5 000,00		337.94	5000.00	337.94
    # 1	2022-03-31	202880		SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ					NaN	NaN
    for i2 in range(0, len2):
        i1 = 2*i2
        r1 = rows[i1]
        r2 = rows[i1+1]    
        row = []
        
        # date = DateTimeField()
        val = dt.datetime.strptime(r1[0] + ' ' + r1[1], '%Y-%m-%d %H:%M')
        row.append(val)

        # date_account = DateTimeField(null=True)    
        val = dt.datetime.strptime(r2[0], '%Y-%m-%d')
        row.append(val)

        # category = CharField(null=True) 
        val = r1[3]
        row.append(val)

        # name = CharField(null=True)
        val = r2[3]
        row.append(val)

        # amount = FloatField()
        st = r1[5]
        val = mm.text_to_float(st) if st[0] == '+' else -mm.text_to_float(st)
        row.append(val)

        # balance
        val = mm.text_to_float(r1[7])
        row.append(val)

        # Сохранить в массив 
        rows2.append(row)

    # res['df']
    # Создать DataFrame из списка rows
    columns=["date", "date_account", "category", "name", "amount", "balance"]
    df = pd.DataFrame(columns=columns, data=rows2).sort_values(by='date')
    res['df'] = df
    res['mess'] = f"Записать/обновить банковскую выписку? \n\n\
            Карта: {res['card_mark']}\n\
            {res['period']}\n\
            Приходы: {res['prop'][3]:_.2f}  \n\
            Расходы: {res['prop'][2]:_.2f}  \n\
            Остаток на начло периода: { res['prop'][0]:_.2f}\n\
            Остаток на конец периода: { res['prop'][1]:_.2f}".replace('_', ' ')
    res['success'] = True
    # df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')

    return res
    
load_payments_from_XML_RNKB(file_name)